In [1]:
import h5py
from tqdm import tqdm, trange

from chainer import cuda
from skimage import img_as_float
from skimage.transform import resize
import numpy as np
import cv2

cuda.Device(2).use()
class Extractor:    
    def __init__(self,model):
        self.model = model
        model.to_gpu()
    def transform(self,f):
        mean = np.array([103.939, 116.779, 123.68])
        img = cv2.imread(f).astype(np.float32)
        img -= mean
        img = cv2.resize(img, (224, 224)).transpose((2, 0, 1))
        img = img[np.newaxis, :, :, :]
        return img
    def get_features(self,fs):
        x = Variable(np.vstack([self.transform(f) for f in fs]), volatile=True)
        x.to_gpu()
        
        fc = vgg(x,None)
        conv = vgg.conv_h
        
        return conv.data.get(),fc.data.get()
    
# https://github.com/mitmul/chainer-imagenet-vgg
from chainer import serializers, Variable
from VGGNet import VGGNet 
vgg = VGGNet()
serializers.load_hdf5('VGG.model', vgg)
extrator = Extractor(vgg)

In [2]:
from tqdm import tqdm, trange
def process_features(filenames, outname, batchsize=32):
    total = len(filenames)
    with h5py.File(outname,'w') as f:
        feats_conv = f.create_dataset('feats_conv', (total,196,512))
        feats_fc = f.create_dataset('feats_fc', (total,4096))
        for i in trange(0,total,batchsize):
            xs = filenames[i:i+batchsize]
            size = len(xs)
            feature_conv, feature_fc = extrator.get_features(xs)
            feats_conv[i:i+size] = feature_conv.reshape(size,512,-1).transpose((0,2,1))
            feats_fc[i:i+size] = feature_fc 

In [3]:
import json
flickr30k = json.load(open('json/dataset_flickr30k.json', 'r'))
train = [item for item in flickr30k['images'] if item['split']=='train']
val = [item for item in flickr30k['images'] if item['split']=='val']
test = [item for item in flickr30k['images'] if item['split']=='test']
data_path = '../data/flickr30k/flickr30k-images/'

In [4]:
process_features([data_path+train[i]['filename'] for i in range(len(train))], 'train_flickr30k.h5', batchsize=128)
process_features([data_path+test[i]['filename'] for i in range(len(test))], 'test_flickr30k.h5', batchsize=128)
process_features([data_path+val[i]['filename'] for i in range(len(val))], 'val_flickr30k.h5', batchsize=128)

100%|██████████| 8/8 [00:16<00:00,  2.00s/it]


In [5]:
import json, h5py
with h5py.File('train_flickr30k.h5') as f:
    print f['feats_conv'].shape
    print f['feats_fc'].shape
    f.attrs['sents'] = unicode(json.dumps(train))
with h5py.File('test_flickr30k.h5') as f:
    print f['feats_conv'].shape
    print f['feats_fc'].shape
    f.attrs['sents'] = unicode(json.dumps(test))
with h5py.File('val_flickr30k.h5') as f:
    print f['feats_conv'].shape
    print f['feats_fc'].shape
    f.attrs['sents'] = unicode(json.dumps(val))

(29000, 196, 512)
(29000, 4096)
(1000, 196, 512)
(1000, 4096)
(1014, 196, 512)
(1014, 4096)
